# Term Paper

### Part 1

In [2]:
import pandas as pd
import glob as glob

In [ ]:
files=glob.glob(f'data/SCE-*-20*.csv')

df_list=[]

waves_number=0

#Loop to read in and append files
for file in files:
    df=pd.read_csv(file,sep=";",parse_dates=['date'])
    df_list.append(df)


sce_df=pd.concat(df_list,ignore_index=True)

# Evaluating unique individuals inside the dataframe 
unique_individuals = sce_df['userid'].nunique() 

# Userid column contains unique identifiers for individuals
print(f"Number of unique individuals: {unique_individuals}")

# Number of rows in the DataFrame 
num_rows = sce_df.shape[0]
print(f"Number of rows in the DataFrame: {num_rows}")

# Setting date as index and sorting
sce_df.set_index('date',inplace=True)
sce_df.sort_index(inplace=True)

# Number of unique survey waves 
unique_waves = sce_df["wid"].nunique()
print(f"Number of unique survey waves: {unique_waves}")

# Finding first and last date in the dataframe, without sorting
first_date = sce_df.index[0]
last_date = sce_df.index[-1]
print(f"First date in the DataFrame: {first_date}")
print(f"Last date in the DataFrame: {last_date}")

Number of unique individuals: 23369
Number of rows in the DataFrame: 176101
Number of unique survey waves: 139
First date in the DataFrame: 2013-06-01 00:00:00
Last date in the DataFrame: 2024-12-31 00:00:00


,userid,wid,weight,female,educ,age,hispanic,black,couple,num_kids,...,num_lit_q3,num_lit_q3_correct,num_lit_q5,num_lit_q5_correct,num_lit_q6,num_lit_q6_correct,num_lit_q8,num_lit_q8_correct,num_lit_q9,num_lit_q9_correct
date,,,,,,,,,,,,,,,,,,,,,
2013-06-01,70027661,201306,2.2,1.0,2.0,54.0,0.0,0.0,1.0,1.0,...,10.0,1.0,100.0,1.0,5.0,1.0,NaN,NaN,NaN,NaN
2013-06-01,70027730,201306,0.3,0.0,3.0,68.0,0.0,0.0,0.0,1.0,...,100.0,0.0,100.0,1.0,5.0,1.0,NaN,NaN,NaN,NaN
2013-06-02,70027707,201306,0.3,1.0,4.0,42.0,0.0,0.0,0.0,2.0,...,10.0,1.0,100.0,1.0,2.0,0.0,NaN,NaN,NaN,NaN
2013-06-02,70011306,201306,1.4,1.0,4.0,37.0,0.0,0.0,1.0,2.0,...,10.0,1.0,10.0,0.0,5.0,1.0,NaN,NaN,NaN,NaN
2013-06-02,70011246,201306,4.2,1.0,3.0,37.0,1.0,1.0,1.0,2.0,...,2.0,0.0,0.0,0.0,5.0,1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31,75023024,202412,0.6,1.0,4.0,46.0,0.0,1.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-12-31,75019318,202412,0.8,0.0,4.0,58.0,0.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-12-31,75022264,202412,1.3,0.0,3.0,63.0,1.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Part 2

In [32]:
# Forward-fill numeracy variables for each individual
numeracy_cols = [
    'num_lit_q1_correct', 'num_lit_q2_correct', 'num_lit_q3_correct',
    'num_lit_q5_correct', 'num_lit_q6_correct', 'num_lit_q8_correct', 'num_lit_q9_correct'
]

# Replace blank strings with NaN (if your CSV uses empty fields)
sce_df[numeracy_cols] = sce_df[numeracy_cols].replace("", pd.NA)


# Group by userid, then forward-fill within each person
sce_df[numeracy_cols] = sce_df.groupby('userid')[numeracy_cols].ffill()

# Drop all rows that still have NaN
numeracy_cols.extend(['female','educ','age','inflation','house_price_change','prob_stocks_up'])
sce_df.dropna(subset=numeracy_cols,inplace=True)

#Eliminate 0.1th percentile and 99.9th percentile of answers
numeracy_cols = [
    'num_lit_q1_correct', 'num_lit_q2_correct', 'num_lit_q3_correct',
    'num_lit_q5_correct', 'num_lit_q6_correct', 'num_lit_q8_correct', 'num_lit_q9_correct'
]
lower_bounds=sce_df[numeracy_cols].quantile(0.001)
upper_bounds=sce_df[numeracy_cols].quantile(0.999)
for col in numeracy_cols:
    sce_df=sce_df[(sce_df[col]>=lower_bounds[col]) & (sce_df[col]<=upper_bounds[col])]

#Creating a column to determine whether the individual has a bachelor's degree or higher
sce_df['college']=(sce_df['educ']>=3).astype(int)

#Compute the total number of correct numeracy responces
sce_df['numeracy_score']=sce_df[numeracy_cols].sum(axis=1)

#Compute the average numeracy score
percentage=sce_df['numeracy_score'].value_counts(normalize=True)*100
percentage.sort_index(inplace=True)
print(f"Numeracy Score Distribution (%): {percentage}")

#Creating a column to determine if the individual has a higher numeracy score than the median
median_score = sce_df['numeracy_score'].median()
sce_df['num_lit_high'] = (sce_df['numeracy_score'] > median_score).astype(int)

Numeracy Score Distribution (%): numeracy_score
0.0     0.122114
1.0     0.659272
2.0     2.001803
3.0     5.189132
4.0     9.538728
5.0    16.398209
6.0    27.388498
7.0    38.702245
Name: proportion, dtype: float64
